## DRAFT: Reading Exported PA Databases

In [ ]:
#%pip install pdfquery
from pdfquery import PDFQuery

path = 'C:\\Users\\casao\Downloads\\Meyer_Peter_19590107_20240327094816.pdf'

pdf = PDFQuery(path)
pdf.load()

# Use CSS-like selectors to locate the elements
text_elements = pdf.pq('LTTextLineHorizontal')

# Extract the text from the elements
text = [t.text for t in text_elements]

print(text)

In [ ]:
# %pip install camelot-py
path = 'C:\\Users\\casao\Downloads\\Meyer_Peter_19590107_20240327094816.pdf'

import camelot.io as camelot
camelot.read_pdf(path)

In [ ]:
import ctypes
from ctypes.util import find_library
find_library("".join(("gsdll", str(ctypes.sizeof(ctypes.c_voidp) * 8), ".dll")))

In [ ]:
%pip install libsqlcipher  


In [ ]:
%pip install sqlcipher3  


In [ ]:
# from: https://stackoverflow.com/questions/986403/encrypted-database-file-in-python

from cryptography.fernet import Fernet
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.backends import default_backend
import base64
# from os import getcwd
import sqlite3
import gzip

def key_creation(password):
    password_bytes = password.encode('utf-8')
    kdf=PBKDF2HMAC(algorithm=hashes.SHA256(), salt=b'\xfaz\xb5\xf2|\xa1z\xa9\xfe\xd1F@1\xaa\x8a\xc2', iterations=1024, length=32, backend=default_backend())
    key=Fernet(base64.urlsafe_b64encode(kdf.derive(password_bytes)))
    return key

def key_creation2(password):
    password_bytes = password.encode('utf-8')
    kdf=PBKDF2HMAC(algorithm=hashes.SHA256(), salt=b'\xfaz\xb5\xf2|\xa1z\xa9\xfe\xd1F@1\xaa\x8a\xc2', iterations=1024, length=32, backend=default_backend())
    key=kdf.derive(password_bytes)
    return key

def encryption(b, password):
    f=key_creation(password)
    safe=f.encrypt(b)
    return safe

def decryption(safe, password):
    f=key_creation(password)
    b=f.decrypt(safe)
    return b

def decryption2(ciphertext, password):
    key = key_creation2(password) # Derive the key from the password
    cipher = Cipher(algorithms.AES(key), modes.CBC(b'\x00' * 16), backend=default_backend()) # Create a cipher object using AES in CBC mode
    decryptor = cipher.decryptor() # Create a decryptor object
    plaintext = decryptor.update(ciphertext) + decryptor.finalize() # Decrypt the ciphertext
    return plaintext

def open_cdb(path,password):
    with open(path, 'rb') as f:
            encrypted_content = f.read()
    # decrypted_content=decryption(encrypted_content,password)
    decrypted_content=decryption2(encrypted_content,password)
    content=decrypted_content.decode('utf-8')
    # content = content.decode('latin-1')
    con=sqlite3.connect(':memory:')
    con.executescript(content)
    return con

def save_cdb(con,path,password):
    fp=gzip.open(path,'wb')
    b=b''
    for line in con.iterdump():
        b+=bytes('%s\n','utf8') % bytes(line,'utf8')
    b=encryption(b,password)
    fp.write(b)
    fp.close()



## trying it out

path = "C:\\Users\\casao\\AppData\\Roaming\\personal-analytics\\exports\\PA_32XSB1_2024-03-04_10-41.sqlite"
pwd = "PersonalAnalytics_32XSB1"


conn = sqlite3.connect(':memory:')
# conn.execute('CREATE TABLE PRODUCTS (ID INT PRIMARY KEY     NOT NULL,\nNAME           TEXT    NOT NULL,\nPRICE            REAL     NOT NULL,\nTAXES        REAL    NOT NULL);')
# save_cdb(conn,name,password)
# conn.close()
conn = open_cdb(path, pwd)
cursor = conn.execute('select * from settings;')
headers = list(map(lambda x: x[0], cursor.description))
print(headers)
for x in cursor:
    for j in range(len(x)):
        print(headers[j]+' ',x[j])
    print('\n')
conn.close()

In [ ]:
%pip install pycryptodome

In [ ]:
# temp ZIP

# %pip install pycryptodome

path = "C:\\TEMP\\PA\\PA_32XSB1.encrypted.zip"
pwd = "PersonalAnalytics_32XSB1"


import zipfile
from Crypto.Cipher import AES
from Crypto.Util.Padding import unpad
import io

def decrypt_aes(key, ciphertext):
    cipher = AES.new(key, AES.MODE_CBC, iv=ciphertext[:16])
    plaintext = cipher.decrypt(ciphertext[16:])
    return unpad(plaintext, AES.block_size)

def extract_encrypted_zip(zip_path, password):
    with open(zip_path, 'rb') as f:
        encrypted_zip = f.read()

    # Assume the first 32 bytes of the encrypted zip file contains the AES key
    key = encrypted_zip[:32]
    encrypted_data = encrypted_zip[32:]

    decrypted_data = decrypt_aes(key, encrypted_data)

    # Use io.BytesIO to create a file-like object from the decrypted data
    decrypted_zip = io.BytesIO(decrypted_data)

    with zipfile.ZipFile(decrypted_zip, 'r') as z:
        z.extractall(pwd=password.encode())

# Usage example:

extract_encrypted_zip(path, pwd)


In [ ]:
%pip install pyzipper

In [ ]:
import sqlite3
import pyzipper
import io

subject = "32XSB1"
zipPath = "C:\\TEMP\\PA\\PA_32XSB1.encrypted.zip"
pwd = "PersonalAnalytics_32XSB1"
dbfile = "database.sqlite"
tableName = "user_input"

charset = "iso-8859-1" # error: 'near "SQLite": syntax error'
# charset = "utf-8" # error: 'utf-8' codec can't decode byte 0xe7 in position 27: invalid continuation byte

with pyzipper.AESZipFile(zipPath) as zf:
    # try 1: extracting then opening the file (in explorer) works
    # zf.extractall(path='C:\\TEMP\\PA\\PA_32XSB1_decrypted', pwd = bytes(pwd, 'utf-8'))
    # print(zf.infolist())

    # try 2: read log/txt file from encrypted zip file works

    # try 3: read sqlite file from encrypted zip file .... :( 
    for file_info in zf.infolist():
            print(file_info)
            if file_info.filename == dbfile:
                with zf.open(file_info, pwd=bytes(pwd, charset)) as sqlite_file:
                     
                    sqlite_data = sqlite_file.read()
                    conn = sqlite3.connect(':memory:')
                    cursor = conn.cursor()
                    sqlite_data_str = sqlite_data.decode(charset)

                    # print(sqlite_data_str) # printing the string actually shows decrypted content

                    # bug: 'near "SQLite": syntax error'
                    cursor.executescript(sqlite_data_str) # conn.executescript(sqlite_data_str) # same issue

                    # Query to list all tables in the database
                    cursor.execute("SELECT name FROM external.sqlite_master WHERE type='table';")

                    # Fetch and print the table names
                    tables = cursor.fetchall()
                    for table in tables:
                        print(f"Table: {table[0]}")
                        cursor.execute(f"PRAGMA table_info(external.{table[0]});")
                        columns = cursor.fetchall()
                        print("Columns:")
                        for column in columns:
                            print(column)

                    conn.close()


In [ ]:
%pip install pysqlcipher3

In [ ]:
# melih's approach

import pysqlcipher3.dbapi2 as sqlcipher

subject = "32XSB1"
path = "C:\\TEMP\\database.sqlite"
pwd = "PersonalAnalytics_32XSB1"
dbfile = "database.sqlite"
tableName = "user_input"

# Create a new database connection and cursor
conn = sqlcipher.connect(path)
cursor = conn.cursor()

# Decrypt the database with the passphrase
cursor.execute("PRAGMA key = '%s'" % pwd)


# Create a test table if it doesn't exist
is_table_exists = cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='test'")
